In [ ]:
!pip install requests
!pip install BeautifulSoup4

In [2]:
# Scraping
import requests
from bs4 import BeautifulSoup

# Data manipulation
import pandas as pd

In [3]:
player_df = pd.DataFrame(columns=[
    '選手名',  # Player Name
    'チーム名',  # Team Name
    '出身地',  # Birthplace
    '国籍',  # Nationality
    '生年月日（満年齢)',  # Date of Birth (Age)
    '身長',  # Height
    '体重',  # Weight
    '過去の所属'  # Past Clubs
])

In [4]:
player_df

,選手名,チーム名,出身地,国籍,生年月日（満年齢),身長,体重,過去の所属


In [5]:
url = 'https://soccer.yahoo.co.jp/jleague/category/j1/teams'
r = requests.get(url)
soup = BeautifulSoup(r.text,'html.parser')
j1teams = soup.select('a[href*="players?"]')

In [6]:
for team in j1teams:

  # Extract and complete each team's URL.
  link = team.get("href")
  complete_link = "https://soccer.yahoo.co.jp" + link

  # Request the team page's HTML.
  request_team = requests.get(complete_link)
  soup = BeautifulSoup(request_team.text,'html.parser')

  # Find links of each player in the team excluding the last 5 players (from scoring ranking).
  player_links = soup.select('a[href*="https://soccer.yahoo.co.jp/jleague/category/j1/teams/"]')[:-5]


  for player_link in player_links:

    # Extract and request each player's page HTML.
    player = player_link.get('href')
    rrr = requests.get(player)
    soup = BeautifulSoup(rrr.text,'html.parser')

    answerItems = soup.find_all('dd', class_='sc-profile__text')

    # Extract each piece of player data from the page.
    player_name = soup.select_one("[class='sc-profile__name']").text.replace('\n',"")
    team_name = soup.select_one("[class='sc-teamHeader__name']").text
    birthplace = answerItems[0].text
    nationality = answerItems[1].text
    date_of_birth = answerItems[2].text
    height = answerItems[3].text
    weight = answerItems[4].text
    previous_club = answerItems[5].text

    # If previous club is one of ['A', 'B', 'O', 'AB'], use next item.
    if previous_club in ['A', 'B', 'O', 'AB']:
      previous_club = answerItems[6].text
    else:
      previous_club = answerItems[5].text


    player_details = pd.DataFrame([{"選手名":player_name,
                                    "チーム名":team_name,
                                    "出身地":birthplace,
                                    "国籍":nationality,
                                    "生年月日（満年齢)": date_of_birth,
                                    "身長":height,
                                    "体重":weight,
                                    "過去の所属":previous_club}], index=[0])

    player_df = pd.concat([player_df, player_details], ignore_index=True)

In [7]:
player_df

,選手名,チーム名,出身地,国籍,生年月日（満年齢),身長,体重,過去の所属
0,菅野 孝憲,北海道コンサドーレ札幌,埼玉,日本,1984年5月3日（39歳）,179cm,75kg,ヴェルディJrユース-東京ヴェルディ1969ユース-横浜FC-柏レイソル-京都サンガF.C....
1,松原 修平,北海道コンサドーレ札幌,北海道,日本,1992年8月11日（30歳）,185cm,87kg,函館シーガル-コンサドーレ札幌ユース・U-15-コンサドーレ札幌ユース・U-18-ファジアー...
2,ク ソンユン,北海道コンサドーレ札幌,韓国,韓国,1994年6月27日（29歳）,195cm,82kg,在鉉中/大韓民国-在鉉高/大韓民国-セレッソ大阪U-18-セレッソ大阪-北海道コンサドーレ札...
3,大谷 幸輝,北海道コンサドーレ札幌,熊本,日本,1989年4月8日（34歳）,186cm,87kg,タイケンSC熊本-ブレイズ熊本-リベルタ北熊本-ベアーズFCエスパーダ-ランザ熊本-浦和レッ...
4,田中 駿汰,北海道コンサドーレ札幌,大阪,日本,1997年5月26日（26歳）,183cm,73kg,NSC北斗SC-ガンバ大阪Jrユース-履正社高-大阪体育大
...,...,...,...,...,...,...,...,...
610,岩崎 悠人,サガン鳥栖,滋賀,日本,1998年6月11日（25歳）,173cm,72kg,金城JFC-JFAアカデミー福島U-15-中央中-京都橘高-京都サンガF.C.-北海道コンサ...
611,横山 歩夢,サガン鳥栖,東京,日本,2003年3月4日（20歳）,170cm,65kg,FCトッカーノU-12-FCトッカーノU-15-東海大高輪台高-松本山雅FC
612,樺山 諒乃介,サガン鳥栖,大阪,日本,2002年9月17日（20歳）,171cm,68kg,SSクリエイト-リップエースSCJrユース-興国高-横浜F・マリノス-モンテディオ山形-横浜...
613,堺屋 佳介,サガン鳥栖,福岡,日本,2005年5月7日（18歳）,173cm,68kg,鯰田FC-サガン鳥栖U-15


In [8]:
#player_df.to_excel("player_information.xlsx", index=False)